In [1]:
%reload_ext setup

INFO:setup:IPython environment configured successfully!


## Introduction to Agents

In [ ]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agent(state_path="model_config.yaml")
# display(agents)

# Cell 3: Run the chat with simplified output
message = "Hello, tell a joke?"
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):  # Only show actual text messages between agents
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):  # Show final result
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

user: Hello, tell a joke?
matcher: Sure! Why don't scientists trust atoms?

Because they make up everything!
critic: This joke is a classic play on words with a clever twist. It's simple and somewhat expected, but it's a great introduction to scientific humor. To enhance your joke-telling, you might consider tailoring the joke to align with current trends or specific interests of the person you're sharing it with, making it feel more personal and engaging. If you have any other jokes or need help tweaking a joke to fit an occasion, feel free to ask!
matcher: Thank you for the feedback! I appreciate your insight. Let me know if you'd like to hear another joke or if you have a specific request.
critic: APPROVE
Task completed. Stop reason: Text 'APPROVE' mentioned


## Demo Example (20 registrants)

In [2]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agents(state_path="model_config.yaml")

# Cell 3: Run the chat with matching task
message = (
    "Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. "
    "Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), "
    "and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. "
    "Output matches in JSON format as per the matching rules."
)
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

user: Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. Output matches in JSON format as per the matching rules.


INFO:openai._base_client:Retrying request to /chat/completions in 30.985000 seconds


matcher: To address resources, I'll start by extracting key mappings between each registration and the potential matching suppliers, considering the rules of matching based on the zip codes and needed products. Subsequently, I'll define JSON objects to clearly outline matches, ensuring each registration is handled appropriately per capacity and distribution rules. 

Here's the breakdown of steps:
1. Review the service areas and available products for each supplier.
2. Match registrations based on the supplier's service area and the products offered.
3. Implement proportional distribution among suppliers with overlapping service areas.
4. Construct the final JSON format for each match.

Let's initiate this process by matching registrations with suppliers based on their service areas and capacity constraints:
```json
[
  {
    "registration_id": "SPUS62763",
    "campaign_id": "MN1",
    "supplier_id": "iSoalr",
    "panel_name": "Silfab HG",
    "panel_capacity": "410",
    "battery_nam

INFO:openai._base_client:Retrying request to /chat/completions in 32.060000 seconds


matcher: It seems like there was an attempt to access a file that doesn't exist. Let's ensure that all necessary data outputs are stored correctly.

Here is the outcome in JSON format, which you can use directly if needed:

```json
[
  {
    "registration_id": "SPUS62763",
    "campaign_id": "MN1",
    "supplier_id": "iSoalr",
    "panel_name": "Silfab HG",
    "panel_capacity": "410",
    "battery_name": "None",
    "battery_capacity": "0"
  },
  {
    "registration_id": "SPUS55759",
    "campaign_id": "MN1",
    "supplier_id": "All Energy Solar",
    "panel_name": "Jinko 425",
    "panel_capacity": "425",
    "battery_name": "None",
    "battery_capacity": "0"
  },
  {
    "registration_id": "SPUS54838",
    "campaign_id": "MN1",
    "supplier_id": "iSoalr",
    "panel_name": "Silfab HG",
    "panel_capacity": "410",
    "battery_name": "None",
    "battery_capacity": "0"
  },
  {
    "registration_id": "SPUS41940",
    "campaign_id": "MN1",
    "supplier_id": "All Energy Solar",
   

...

critic: To proceed with the task as outlined, we need to perform several verifications and re-assessment of the matches provided by the matcher. Here are the steps I will take to ensure the matches are correctly executed:

1. **Verify ZIP Code Alignment**: I'll check if each registration falls within the service area of the assigned supplier.
2. **Check Supplier Capacity**: Ensure that the supplier capacity has not been exceeded in the assigned matches.
3. **Validate Distribution Rules**: Ensure suppliers with overlapping service areas have a distribution aligned with the relative distribution mentioned.
4. **Verify Product Needs**: Confirm the assigned supplier offers the correct product needed by each registration.

Let me now verify the output against these criteria.
**Feedback for the Matcher**:

1. **Registration 'SPUS61978' assigned to incorrect supplier 'iSoalr'**:
   - The ZIP code '55446' should instead fall under the service area of another supplier according to the matching 

INFO:openai._base_client:Retrying request to /chat/completions in 25.394000 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 49.896000 seconds


critic: Let's address the feedback and improve the matching results:

1. **Verify ZIP Code Alignment and Supplier Service Areas**: Initially, I'll ensure each registration is matched with a supplier that services their ZIP code. For example, any registration with ZIP code '55446' should be appropriately assigned to suppliers within its service area.

2. **Re-evaluate Proportional Distribution**: We'll need to properly balance the distribution considering each supplier's capacity and relative distribution rules. We can do this by evaluating allocations per supplier and comparing against their capacity and relative distribution goals.

3. **Review Product and Supplier Offerings**: Make sure all registrations are matched with suppliers offering the required products (panels and batteries).

I have resolved the issues and here is the updated matching output:

```json
[
  {
    "registration_id": "SPUS62763",
    "campaign_id": "MN1",
    "supplier_id": "iSoalr",
    "panel_name": "Silfab H

INFO:openai._base_client:Retrying request to /chat/completions in 42.258000 seconds


matcher: Here is the revised match output based on improved alignment for ZIP code, capacity distribution, and product fit:

```json
[
  {
    "registration_id": "SPUS62763",
    "campaign_id": "MN1",
    "supplier_id": "iSoalr",
    "panel_name": "Silfab HG",
    "panel_capacity": "410",
    "battery_name": "Tesla Powerwall 2",
    "battery_capacity": "13.5"
  },
  {
    "registration_id": "SPUS55759",
    "campaign_id": "MN1",
    "supplier_id": "All Energy Solar",
    "panel_name": "Jinko 425",
    "panel_capacity": "425",
    "battery_name": "None",
    "battery_capacity": "0"
  },
  {
    "registration_id": "SPUS54838",
    "campaign_id": "MN1",
    "supplier_id": "iSoalr",
    "panel_name": "Silfab HG",
    "panel_capacity": "410",
    "battery_name": "None",
    "battery_capacity": "0"
  },
  {
    "registration_id": "SPUS41940",
    "campaign_id": "MN1",
    "supplier_id": "All Energy Solar",
    "panel_name": "Jinko 425",
    "panel_capacity": "425",
    "battery_name": "None"